In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, BatchNormalization, Flatten, Dropout, RandomFlip, RandomRotation
from tensorflow.keras.layers import Rescaling, Resizing, AveragePooling2D, GlobalAvgPool2D
from tensorflow.keras.activations import relu
from tensorflow.keras.models import Model, Sequential
import numpy as np

In [2]:
from tensorflow.keras.applications import vgg16, vgg19, mobilenet, mobilenet_v2, mobilenet_v3, ResNet50

In [3]:
model = mobilenet_v2.MobileNetV2(include_top = True, input_shape = (224,224,3))

In [4]:
model_ = Model(model.input, outputs = model.layers[-10].output)
model_.trainable = False

In [5]:
model_(np.random.randn(1,224,224,3))

<tf.Tensor: shape=(1, 7, 7, 960), dtype=float32, numpy=
number([[[[0.00000000e+00, 0.00000000e+00, 3.00266695e+00, ...,
          0.00000000e+00, 7.78973335e-03, 7.99900219e-02],
         [0.00000000e+00, 0.00000000e+00, 3.49860263e+00, ...,
          0.00000000e+00, 1.25010004e-02, 1.27201706e-01],
         [0.00000000e+00, 0.00000000e+00, 2.46250963e+00, ...,
          0.00000000e+00, 3.67786246e-03, 0.00000000e+00],
         ...,
         [0.00000000e+00, 0.00000000e+00, 2.21845078e+00, ...,
          0.00000000e+00, 2.70746811e-03, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 3.23680687e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 2.77449155e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

        [[0.00000000e+00, 0.00000000e+00, 3.86490059e+00, ...,
          0.00000000e+00, 1.69080459e-02, 1.11095175e-01],
         [0.00000000e+00, 0.00000000e+00, 4.23909092e+00, ...,
          

In [6]:
model_ = Sequential([
        model_,
        GlobalAvgPool2D(),
        Flatten(),
     Dropout(0.8),
    Dense(102)
])

In [7]:
dataset_n = "oxford_flowers102"
import tensorflow_datasets as tfds
# --- #
# Construct a tf.data.Dataset
# --- #
ds_train, ds_test = tfds.load(dataset_n, split=['train','test'], shuffle_files=True, as_supervised = True)

In [8]:
def transform(image, label):
    image = Resizing(224,224)(image)
    image = Rescaling(1/127.5, offset = -1)(image)
    return image, label
def pipeline(data_set, train = True):
    data_set = data_set.map(transform, num_parallel_calls = tf.data.AUTOTUNE)
    data_set = data_set.cache()
    if train:
        data_set = data_set.shuffle(10000)
    data_set = data_set.batch(16)
    data_set = data_set.prefetch(tf.data.AUTOTUNE)
    return data_set

In [9]:
ds_train = pipeline(ds_train)
ds_test = pipeline(ds_test, False)

In [10]:
model_.compile(optimizer= "adam", loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True), metrics = tf.keras.metrics.SparseCategoricalAccuracy())
model_.fit(ds_train, epochs = 10, validation_data = ds_test) 

Epoch 1/10
64/64 [==============================] - 31s 374ms/step - loss: 8.2761 - sparse_categorical_accuracy: 0.0088 - val_loss: 4.5643 - val_sparse_categorical_accuracy: 0.0218
Epoch 2/10
64/64 [==============================] - 12s 187ms/step - loss: 7.4370 - sparse_categorical_accuracy: 0.0088 - val_loss: 4.2604 - val_sparse_categorical_accuracy: 0.0738
Epoch 3/10
64/64 [==============================] - 9s 150ms/step - loss: 6.8940 - sparse_categorical_accuracy: 0.0196 - val_loss: 4.0075 - val_sparse_categorical_accuracy: 0.1408
Epoch 4/10
64/64 [==============================] - 9s 149ms/step - loss: 6.0194 - sparse_categorical_accuracy: 0.0265 - val_loss: 3.7915 - val_sparse_categorical_accuracy: 0.1984
Epoch 5/10
64/64 [==============================] - 10s 151ms/step - loss: 5.6305 - sparse_categorical_accuracy: 0.0382 - val_loss: 3.5404 - val_sparse_categorical_accuracy: 0.3326
Epoch 6/10
64/64 [==============================] - 10s 152ms/step - loss: 5.0413 - sparse_catego

KeyboardInterrupt: ignored

In [ ]:
y_t_pred =model_.predict()